[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/spacewalk01/yolov8-google-open-images/demo.ipynb)

In [1]:
import pandas as pd
from os import path
import os
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm  

import fiftyone as fo
import fiftyone.zoo as foz

In [8]:
root = os.getcwd()     # set your path
target_classes = ['Tortoise'] # set your classes

num_epochs = 10
batch_size = 16

In [3]:
output_dir = path.join(root, 'open-images-dataset')
dataset_name = 'open-images-v7'
num_classes = len(target_classes)

if not os.path.exists(root):
    os.makedirs(root)

In [5]:
def download_open_images_dataset(split_name, max_samples):
    dataset = foz.load_zoo_dataset(
        dataset_name,
        split=split_name,
        dataset_dir=path.join(root, dataset_name),
        label_types=["detections"],
        classes=target_classes,  
        drop_existing_dataset=True,
        max_samples=max_samples  # None means all examples
    )
    
    return dataset

In [6]:
train_dataset = download_open_images_dataset('train', 1000)
valid_dataset = download_open_images_dataset('validation', 100)
test_dataset  = download_open_images_dataset('test', 100)

Necessary images already downloaded
Existing download of split 'train' is sufficient
Deleting existing dataset 'open-images-v7-train-1000'
Loading 'open-images-v7' split 'train'
 100% |███████████████| 1000/1000 [3.4s elapsed, 0s remaining, 275.5 samples/s]      
Dataset 'open-images-v7-train-1000' created
Only found 40 (<100) samples matching your requirements
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Deleting existing dataset 'open-images-v7-validation-100'
Loading 'open-images-v7' split 'validation'
 100% |███████████████████| 40/40 [165.8ms elapsed, 0s remaining, 241.2 samples/s]     
Dataset 'open-images-v7-validation-100' created
Necessary images already downloaded
Existing download of split 'test' is sufficient
Deleting existing dataset 'open-images-v7-test-100'
Loading 'open-images-v7' split 'test'
 100% |█████████████████| 100/100 [421.3ms elapsed, 0s remaining, 238.6 samples/s]      
Dataset 'open-images-v7-test-100' created


In [10]:
session = fo.launch_app(valid_dataset.view())

In [11]:
def get_image_file_names(directory):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
    image_file_names = []

    for filename in os.listdir(directory):
        file_extension = path.splitext(filename)[1]
        if file_extension in image_extensions:
            image_file_names.append(filename)

    return image_file_names

In [12]:
class_list_filepath = path.join(root, dataset_name, 'train', 'metadata/classes.csv')
class_df = pd.read_csv(class_list_filepath, header=None, names=['URI', 'ClassName'])
class_map = dict(zip(class_df.ClassName, class_df.URI))
class_map = dict([(class_map[cls], idx) for idx, cls in enumerate(target_classes)])

In [13]:
def load_labels(split_name):
    labels_filepath = path.join(root, dataset_name, split_name, 'labels/detections.csv')
    df = pd.read_csv(labels_filepath) 
    return df

train_df = load_labels('train')    
valid_df = load_labels('validation')    
test_df = load_labels('test') 

valid_df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,0001eeaf4aed83f9,xclick,/m/0cmf2,1,0.022673,0.964201,0.071038,0.800546,0,0,0,0,0
1,000595fe6fee6369,xclick,/m/02wbm,1,0.000000,1.000000,0.000000,1.000000,0,0,1,0,0
2,000595fe6fee6369,xclick,/m/02xwb,1,0.141384,0.179676,0.676275,0.731707,0,0,0,0,0
3,000595fe6fee6369,xclick,/m/02xwb,1,0.213549,0.253314,0.299335,0.354767,1,0,0,0,0
4,000595fe6fee6369,xclick,/m/02xwb,1,0.232695,0.288660,0.490022,0.545455,1,0,0,0,0


In [14]:
def convert_to_yolo(df, split_name):
    
    images_dir = path.join(root, dataset_name, split_name, 'data')
    image_filenames = get_image_file_names(images_dir)
    image_ids = set([path.splitext(filename)[0] for filename in image_filenames])
    
    df = df[df['ImageID'].isin(image_ids)] 
    df = df[df['LabelName'].isin(class_map.keys())] 

    # YOLO FORMAT: Each row is class x_center y_center width height format.
    df['Width']  = df['XMax'] - df['XMin']
    df['Height'] = df['YMax'] - df['YMin']
    df['X'] = df['XMin'] + df['Width'] / 2.0 
    df['Y'] = df['YMin'] + df['Height'] / 2.0

    df['ClassId'] = df['LabelName'].map(class_map)
    return df

train_df = convert_to_yolo(train_df, 'train')
valid_df = convert_to_yolo(valid_df, 'validation')
test_df = convert_to_yolo(test_df, 'test')

In [15]:
valid_df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,Width,Height,X,Y,ClassId
3013,0231f3d9cfa79b30,xclick,/m/011k07,1,0.106990,0.467903,0.365297,0.563927,0,0,0,0,0,0.360913,0.198630,0.287447,0.464612,0
3014,0231f3d9cfa79b30,xclick,/m/011k07,1,0.420827,0.844508,0.527397,0.783105,0,0,0,0,0,0.423680,0.255708,0.632668,0.655251,0
5143,04728d00324ff2af,xclick,/m/011k07,1,0.000000,0.678125,0.377083,0.812500,0,0,0,0,0,0.678125,0.435417,0.339062,0.594792,0
6885,05fb96a824503d43,xclick,/m/011k07,1,0.167870,0.880866,0.314079,0.740072,0,0,0,0,0,0.712996,0.425993,0.524368,0.527076,0
8232,06c9c912e57d0846,xclick,/m/011k07,1,0.000000,0.287500,0.364583,0.660417,0,0,0,0,0,0.287500,0.295833,0.143750,0.512500,0


In [16]:
COLUMNS_TO_DROP = ['XMin','XMax','YMin','YMax','Source','LabelName','Confidence',
                   'IsOccluded','IsTruncated','IsGroupOf','IsDepiction','IsInside', 
                   'XClick1X','XClick2X','XClick3X','XClick4X','XClick1Y','XClick2Y','XClick3Y','XClick4Y']

def drop_unused_cols(df):
    for column_name in COLUMNS_TO_DROP:
        if column_name in df.columns:
            df.drop(columns=column_name, inplace=True)
    return df

train_df = drop_unused_cols(train_df)
valid_df = drop_unused_cols(valid_df)
test_df = drop_unused_cols(test_df)

In [19]:
def write_series_to_txt(series, split_name):
    output_dir = path.join(root, dataset_name, split_name, 'data')
    label_path = path.join(output_dir, '{}.txt'.format(series['ImageID'].iloc[0]))
    series[['ClassId', 'X', 'Y', 'Width', 'Height']].to_csv(label_path, index=False, header=False, sep=' ')

def applyParallel(grouped_df, func, split_name):
    num_cpus = multiprocessing.cpu_count()
    Parallel(n_jobs=num_cpus)(delayed(func)(group, split_name) for name, group in tqdm(grouped_df))
    
applyParallel(train_df.groupby(train_df.ImageID), write_series_to_txt, 'train')
applyParallel(valid_df.groupby(valid_df.ImageID), write_series_to_txt, 'validation')
applyParallel(test_df.groupby(test_df.ImageID), write_series_to_txt, 'test')


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2500.02it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 2352.75it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1429.12it/s]


In [23]:
train_filenames = get_image_file_names(path.join(root, dataset_name, 'train', 'data'))
valid_filenames = get_image_file_names(path.join(root, dataset_name, 'validation', 'data'))
test_filenames = get_image_file_names(path.join(root, dataset_name, 'test', 'data'))

In [30]:
import yaml

def create_filename_list(filenames, split_name):
    with open(path.join(root , split_name + '.txt'), 'w') as f:
        for filename in filenames:
            f.write(path.join(root, dataset_name, split_name, 'data', filename)+'\n')
            
create_filename_list(train_filenames, 'train')
create_filename_list(valid_filenames, 'validation')
create_filename_list(test_filenames, 'test')

data = dict(
    path  = root,
    train = 'train.txt',
    val   = 'validation.txt',
    test  = 'test.txt',
    nc    = len(target_classes),
    names = target_classes,
    )

with open(os.path.join( root , 'custom.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join(root, 'custom.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- Tortoise
nc: 1
path: E:/open-images
test: test.txt
train: train.txt
val: validation.txt



In [10]:
!yolo task=detect mode=train model=yolov8n.pt data=custom.yaml epochs={num_epochs} imgsz=640 batch={batch_size}

New https://pypi.org/project/ultralytics/8.0.147 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.72  Python-3.7.13 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=custom.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_ma